In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install --upgrade nbformat


In [ ]:
%matplotlib inline


In [ ]:
!pip install umap-learn[plot]

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import umap.plot
import umap.umap_ as umap
from collections import defaultdict
import matplotlib.pyplot as plt
from ml4h.explorations import latent_space_dataframe,stratify_and_project_latent_space,plot_nested_dictionary,iterative_subspace_removal

In [ ]:
#code to copy files from google cloud
# !gsutil cp gs://fc-e5ea247f-18af-45e4-825a-71f0537e3cb7/hidden_ecg_rest_median_raw_10_dropout_pair_contrastive_lax_4ch_cycle_ecg_median_10_pretrained_256d_v2020_06_07.tsv latent_file.tsv
    

# !gsutil cp gs://fc-e5ea247f-18af-45e4-825a-71f0537e3cb7/tensors_all_union.csv tensors_all_union.csv
    
# !gsutil cp gs://fc-e5ea247f-18af-45e4-825a-71f0537e3cb7/hidden_lax_4ch_heart_center_dropout_pair_contrastive_lax_4ch_cycle_ecg_median_10_pretrained_256d_v2020_06_07.tsv latent_file_for_comparison.tsv

In [ ]:
# Loading file path from local dir, gsutil cp was not working but you can try gsutil
ml4h_dir = os.path.dirname(os.getcwd())
latent_file = os.path.join(ml4h_dir,"latent_data_test/latent_file.tsv")
latent_file_for_comparison = os.path.join(ml4h_dir,"latent_data_test/latent_file_for_comparison.tsv")
label_file = os.path.join(ml4h_dir,"latent_data_test/tensors_all_union.csv") 

In [ ]:
# defining categorical and continuous covariates variables
categorical_covariates = ['Sex_Male_0_0',
 'atrial_fibrillation_or_flutter',
 'coronary_artery_disease',
 'diabetes_type_2',
 'hypercholesterolemia',
 'hypertension',
 'Sinus_bradycardia',
 'Marked_sinus_bradycardia',
 'Other_sinus_rhythm']

continuous_covariates = ['age', 'bmi', 'RRInterval', 'QTInterval', 'LVM', 'RVEDV', 'PC1', 'PC2', 'PC3']

all_scores = defaultdict(dict)


In [ ]:
continuous_covariates

In [ ]:
#renaming columns of latent file
col_rename = {f'22009_Genetic-principal-components_0_{i}': f'PC{i}' for i in range(1,41)}
col_rename['Genetic-sex_Male_0_0'] = 'sex'
col_rename['21003_Age-when-attended-assessment-centre_2_0'] = 'age'
col_rename['21001_Body-mass-index-BMI_2_0'] = 'bmi'

# ECG Inferences from contrasive paired model

In [ ]:
latent_df = latent_space_dataframe(latent_file, label_file)
latent_df = latent_df.rename(columns=col_rename)
latent_df.head(5)

In [ ]:
latent_dimension = 256
latent_cols = [f'latent_{i}' for i in range(latent_dimension)]
for c in categorical_covariates:
    all_scores['DropFuse ECG'].update(stratify_and_project_latent_space(c, 1, 0, latent_cols, latent_df))
for c in continuous_covariates:
    all_scores['DropFuse ECG'].update(stratify_and_project_latent_space(c, latent_df[c].median(), 
                                                                        0, latent_cols, latent_df))
all_scores    

In [ ]:
plot_nested_dictionary(all_scores)

# MRI Inferences from contrasive paired model

In [ ]:
# loading latent space file
latent_df = latent_space_dataframe(latent_file_for_comparison, label_file)
latent_dimension = 256
latent_cols = [f'latent_{i}' for i in range(latent_dimension)]
latent_df = latent_df.rename(columns=col_rename)
for c in categorical_covariates:
    all_scores['DropFuse MRI'].update(stratify_and_project_latent_space(c, 1, 0, latent_cols, latent_df))
for c in continuous_covariates:
    all_scores['DropFuse MRI'].update(stratify_and_project_latent_space(c, latent_df[c].median(), 
                                                                        0, latent_cols, latent_df))


In [ ]:
plot_nested_dictionary(all_scores)

In [ ]:
space = latent_df[latent_cols].to_numpy()
umapped = umap.UMAP().fit_transform(space)
print(umapped.shape)
print(latent_df['Sex_Male_0_0'].to_numpy().shape)

In [ ]:
fig, ax = plt.subplots(figsize=(3,3), dpi=300)
ax.scatter(umapped[latent_df['Sex_Male_0_0'].to_numpy()==1,0], 
           umapped[latent_df['Sex_Male_0_0'].to_numpy()==1,1], s=.1, label='Male')
ax.scatter(umapped[latent_df['Sex_Male_0_0'].to_numpy()==0,0], 
           umapped[latent_df['Sex_Male_0_0'].to_numpy()==0,1], s=.1, label='Female')
ax.axis("off")
ax.legend()

In [ ]:
r2_thresh = 0.01
adjust_cols = ['Sex_Male_0_0']
new_cols, latent_df = iterative_subspace_removal(adjust_cols, latent_df, latent_cols, r2_thresh=r2_thresh)

In [ ]:
new_space = latent_df[new_cols].to_numpy()
umapped_debias = umap.UMAP().fit_transform(new_space)

In [ ]:
fig, ax = plt.subplots(figsize=(3,3), dpi=300)
ax.scatter(umapped_debias[latent_df['Sex_Male_0_0'].to_numpy()==1,0], 
           umapped_debias[latent_df['Sex_Male_0_0'].to_numpy()==1,1], s=.1, label='Male')
ax.scatter(umapped_debias[latent_df['Sex_Male_0_0'].to_numpy()==0,0], 
           umapped_debias[latent_df['Sex_Male_0_0'].to_numpy()==0,1], s=.1, label='Female')
ax.axis("off")
ax.legend()